# Prerequirements

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import PIL
import cv2
import glob

import tensorflow as tf
import tensorflow_datasets as tfds
from tensorflow import keras
from tensorflow.keras.layers import Input, Conv2D, MaxPooling2D, Conv2DTranspose, BatchNormalization, Layer, LeakyReLU

import warnings
warnings.filterwarnings("ignore")

In [ ]:
physical_devices = tf.config.list_physical_devices('GPU')
print(physical_devices)
if len(physical_devices) > 0:
    tf.config.experimental.set_memory_growth(physical_devices[0], True)

In [ ]:
def imshow(img: np.array):
    if img.shape[0] * 2 > img.shape[1]:
        fig = plt.figure(figsize=(7, 7))
    else:
        fig = plt.figure(figsize=(20, 20))
    plt.axis('off')
    plt.imshow(img)

# Preparing dataset

In [ ]:
ds = []
for idx, file in enumerate(glob.glob("/kaggle/input/celeba-dataset/img_align_celeba/img_align_celeba/*")):
    ds.append(np.array(PIL.Image.open(file).resize((112, 96))))
    if idx >= 10001: break
ds = np.array(ds)
ds.shape

In [ ]:
idxs = np.random.choice(len(ds), 5)
imshow(np.concatenate(ds[idxs], 1))

In [ ]:
def create_line_mask(img):
    mask = np.full(img.shape, 255, np.uint8)
    for _ in range(np.random.randint(6, 10)):
        x1, x2 = np.random.randint(1, img.shape[1]), np.random.randint(1, img.shape[1])
        y1, y2 = np.random.randint(1, img.shape[0]), np.random.randint(1, img.shape[0])
        thickness = np.random.randint(4, 6)
        cv2.line(mask, (x1, y1), (x2, y2), (1, 1, 1), thickness)

    masked_image = cv2.bitwise_and(img, mask)

    return masked_image

In [ ]:
idxs = np.random.choice(len(ds), 5)
masked = np.array(list(map(create_line_mask, ds[idxs])))
imshow(np.concatenate(masked, 1))

In [ ]:
! mkdir ./data
! mkdir ./data/samples
! mkdir ./data/samples_line_masked
! mkdir ./data/samples_square_masked

In [ ]:
for idx, sample in enumerate(ds):
    PIL.Image.fromarray(sample).save(f'./data/samples/{idx}.png')
    PIL.Image.fromarray(create_line_mask(sample)).save(f'./data/samples_line_masked/{idx}.png')
    PIL.Image.fromarray(create_line_mask(sample)).save(f'./data/samples_square_masked/{idx}.png')

# Working in Kaggle, output refreshes when tourn off, so created dataset cv3-data/data...

# Custom generator

In [ ]:
# class DataGenerator(keras.utils.Sequence):
#     def __init__(self, X, Y, batch_size=64, dim=(208, 176), n_channels=3): 
#         self.X = X
#         self.Y = Y
#         self.batch_size = batch_size
#         self.dim = dim
#         self.n_channels = n_channels

#         self.on_epoch_end()
#         assert(len(self.X) == len(self.Y) or len(self.X) > 0)

        
#     def __len__(self):
#         return int(np.floor(len(self.X) / self.batch_size))


#     def __getitem__(self, index):
#         indexes = self.indexes[index * self.batch_size : (index+1) * self.batch_size]
#         return self.__data_generation(indexes)

    
#     def on_epoch_end(self):
#         self.indexes = np.arange(len(self.X))
    

#     def __data_generation(self, idxs):
#         X_batch = np.empty((self.batch_size, self.dim[0], self.dim[1], self.n_channels))
#         Y_batch = np.empty((self.batch_size, self.dim[0], self.dim[1], self.n_channels))

#         for i, idx in enumerate(idxs):
#             image = np.array(PIL.Image.open(self.X[idx]))
#             label = np.array(PIL.Image.open(self.Y[idx]))
            
#             X_batch[i,] = image / 255
#             Y_batch[i,] = label / 255

#         return X_batch, Y_batch

In [ ]:
# def train_test_split(X, Y, train_size=0.8):   
#     train_split = int(train_size * len(X))
    
#     X_train = X[:train_split]
#     Y_train = Y[:train_split]
    
#     X_test = X[train_split:]
#     Y_test = Y[train_split:]
    
#     return X_train, X_test, Y_train, Y_test

In [ ]:
# X = sorted(glob.glob("/kaggle/input/cv3-data/data/samples_square_masked/*.png"))
# Y = sorted(glob.glob("/kaggle/input/cv3-data/data/samples/*.png"))

# X_train, X_test, Y_train, Y_test = train_test_split(X, Y)

In [ ]:
# train_gen = DataGenerator(X_train, Y_train)
# test_gen = DataGenerator(X_test, Y_test)

In [ ]:
# class inpaintingModel:
#   '''
#   Build UNET like model for image inpaining task.
#   '''
#   def prepare_model(self, input_size=(208, 176, 3)):
#     inputs = keras.layers.Input(input_size)

#     conv1, pool1 = self.__ConvBlock(32, (3,3), (2,2), 'relu', 'same', inputs) 
#     conv2, pool2 = self.__ConvBlock(64, (3,3), (2,2), 'relu', 'same', pool1)
#     conv3, pool3 = self.__ConvBlock(128, (3,3), (2,2), 'relu', 'same', pool2) 
#     conv4, pool4 = self.__ConvBlock(256, (3,3), (2,2), 'relu', 'same', pool3) 
    
#     conv5, up6 = self.__UpConvBlock(512, 256, (3,3), (2,2), (2,2), 'relu', 'same', pool4, conv4)
#     conv6, up7 = self.__UpConvBlock(256, 128, (3,3), (2,2), (2,2), 'relu', 'same', up6, conv3)
#     conv7, up8 = self.__UpConvBlock(128, 64, (3,3), (2,2), (2,2), 'relu', 'same', up7, conv2)
#     conv8, up9 = self.__UpConvBlock(64, 32, (3,3), (2,2), (2,2), 'relu', 'same', up8, conv1)
    
#     conv9 = self.__ConvBlock(32, (3,3), (2,2), 'relu', 'same', up9, False)
    
#     outputs = keras.layers.Conv2D(3, (3, 3), activation='sigmoid', padding='same')(conv9)

#     return keras.models.Model(inputs=[inputs], outputs=[outputs])  

#   def __ConvBlock(self, filters, kernel_size, pool_size, activation, padding, connecting_layer, pool_layer=True):
#     conv = keras.layers.Conv2D(filters=filters, kernel_size=kernel_size, activation=activation, padding=padding)(connecting_layer)
#     conv = keras.layers.Conv2D(filters=filters, kernel_size=kernel_size, activation=activation, padding=padding)(conv)
#     if pool_layer:
#       pool = keras.layers.MaxPooling2D(pool_size)(conv)
#       return conv, pool
#     else:
#       return conv

#   def __UpConvBlock(self, filters, up_filters, kernel_size, up_kernel, up_stride, activation, padding, connecting_layer, shared_layer):
#     conv = keras.layers.Conv2D(filters=filters, kernel_size=kernel_size, activation=activation, padding=padding)(connecting_layer)
#     conv = keras.layers.Conv2D(filters=filters, kernel_size=kernel_size, activation=activation, padding=padding)(conv)
#     up = keras.layers.Conv2DTranspose(filters=up_filters, kernel_size=up_kernel, strides=up_stride, padding=padding)(conv)
#     up = keras.layers.concatenate([up, shared_layer], axis=3)

#     return conv, up

In [ ]:
# model = inpaintingModel().prepare_model(input_size=(208, 176, 3))
# model.compile(optimizer='adam', loss='mean_absolute_error')

# history = model.fit(train_gen, 
#           validation_data=test_gen, 
#           epochs=20, 
#           steps_per_epoch=len(train_gen), 
#           validation_steps=len(test_gen),
#           use_multiprocessing=True)

In [ ]:
# plt.imshow(model.predict(np.array([test_gen[1][0][61]])).reshape(208, 176, 3))